<a href="https://colab.research.google.com/github/ryankemmer/FallDetection/blob/master/FallDetection_Pose_Based_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Initialize Runtime**

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#initialize array of categoires to be included in data
categories = ['Coffee_room_01','Home_01','validation','test']

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Build Openpose (Run Once)

In [0]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`


#Preprocess Data (Run Once)

**Determine categories to be preprocessed (optional)**

In [0]:
categories = ['test2']

**Convert videos to images (run once)**

In [0]:
import os
import cv2
import math 
import glob

#Move to fall detection folder
os.chdir('/content/drive/My Drive/Fall Detection')

#make testdata directory for each category

for i in categories:
  os.mkdir(os.path.join(str(i),'testdata'))
  
#for every category, loop through videos and turn them into pictures
for i in categories:

  path = str(i) + '/Videos/'
  list = os.listdir(path) 
  numvideos = len(list)
  j = 0
  while (j < numvideos):
    print("Writing Video: " + str(j))
    
    if(str(i) == 'validation' and j == 10):
      break
    
    os.mkdir(str(i) + '/testdata' + '/video' + str(j)) # make new directory for each video
    vidcap = cv2.VideoCapture(str(i) + '/Videos/' +'video (' + str(j+1) + ').avi')
    success,image = vidcap.read()
    count = 0
    success = True
    path2 = str(i) + '/testdata' + '/video' + str(j)
    while success:
      cv2.imwrite(os.path.join(path2 , "frame%d.jpg" % count), image)
      success,image = vidcap.read()
      count += 1
    j+=1

Writing Video: 0
Writing Video: 1
Writing Video: 2
Writing Video: 3
Writing Video: 4
Writing Video: 5
Writing Video: 6
Writing Video: 7
Writing Video: 8
Writing Video: 9


**Build openpose libarary to process images**



**Run help reference to ensure libraries are correctly installed**

In [0]:
!cd openpose && ./build/examples/openpose/openpose.bin --help

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
chdir: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
/bin/bash: line 0: cd: openpose: Transport endpoint is not connected


**Turn images into json files with pose data (20 videos per category) (run once)**

In [0]:
import os
import cv2
import math 
import glob

for i in categories:
     
  print("Entering folder: " + str(i))
  os.chdir('/content/drive/My Drive/Fall Detection')
  os.mkdir(os.path.join(str(i),'jsondata'))
  category = str(i)
  testpath = str(i) + '/testdata/'
  list = os.listdir(testpath) 
  numvideos = len(list)
  j = 0
  while (j < numvideos):
    print("Rendering video: " + str(j))
    os.chdir('/content/drive/My Drive/Fall Detection')
    os.mkdir(str(i) + '/jsondata' + '/video' + str(j)) # make new directory for each video
    video = str(j)
    os.chdir('/content')
    !cd openpose && ./build/examples/openpose/openpose.bin -image_dir ../drive/My\ Drive/Fall\ Detection/$category/testdata/video$video  --render_pose 0 -keypoint_scale 3 --display 0 -write_json ../drive/My\ Drive/Fall\ Detection/$category/jsondata/video$video
    j += 1


Entering folder: test2
Rendering video: 0
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 22.633570 seconds.
Rendering video: 1
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 33.126619 seconds.
Rendering video: 2
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 39.459350 seconds.
Rendering video: 3
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total ti

**Fix json files to be read correctly (only run once)**

In [0]:
os.chdir('/content/drive/My Drive/Fall Detection')

for i in categories:
  
  testpath = str(i) + '/testdata/'
  list = os.listdir(testpath) 
  numvideos = len(list)
  j = 0
  while(j < numvideos):
    print("Rewriting json datafrom folder:" + str(i) + "Video: " + str(j))
    if(str(i) == 'validation' and j == 11):
      break
      
    testpath = '/content/drive/My Drive/Fall Detection/' + str(i) + '/jsondata/video' + str(j)
    list = os.listdir(testpath) 
    numframes = len(list)
    k = 0 
    while(k < numframes):
      with open('/content/drive/My Drive/Fall Detection/' + str(i) + '/jsondata/video' + str(j) + '/frame' + str(k) + '_keypoints.json', "a") as f:
        f.write("\n]")
      with open('/content/drive/My Drive/Fall Detection/' + str(i) + '/jsondata/video' + str(j) + '/frame' + str(k) + '_keypoints.json', "r+") as f:
        content = f.read()
        f.seek(0, 0)
        f.write('[\n' + content)     
      k += 1  
    j += 1

        

Rewriting json datafrom folder:test2Video: 0
Rewriting json datafrom folder:test2Video: 1
Rewriting json datafrom folder:test2Video: 2
Rewriting json datafrom folder:test2Video: 3
Rewriting json datafrom folder:test2Video: 4
Rewriting json datafrom folder:test2Video: 5
Rewriting json datafrom folder:test2Video: 6
Rewriting json datafrom folder:test2Video: 7
Rewriting json datafrom folder:test2Video: 8
Rewriting json datafrom folder:test2Video: 9


# **Load in Training Data**

In [0]:
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import numpy as np
from PIL import Image
from keras.utils import np_utils
from skimage.transform import resize
import pandas as pd
import os
import glob
import numpy as np
from keras.utils import np_utils
from keras.utils import to_categorical
import json

#count successful poses
nullposes = 0
successfulposes = 0

#total videos
TotalTrain = 40
TotalTest = 10

#number of samples per activity (walking, falling, laying down)
A = 20
#of frames per sample (timesteps)
B = 30

X_train = np.zeros((TotalTrain*A*2,B,75)) #creating an empty array for image json data
X_test = np.zeros((TotalTest*A*2,B,75))

Y_train = np.zeros((TotalTrain*A*2,1)) #create empty array for y data
Y_test = np.zeros((TotalTest*A*2,1))

#initialize counters
trainframecount = 0
testframecount = 0

#loop through categories
for i in categories:
  
  if(str(i) == 'test'):
    break
  
  #count the number of videos in category
  path = '/content/drive/My Drive/Fall Detection/' + str(i) + '/Annotation_files'
  list = os.listdir(path)
  numvideos = len(list)
  
  #change into annotations file folder
  print("Entering folder " + str(i) + ", which has " + str(numvideos) + " videos to process")
  os.chdir(path)
  
  #separate if test dataset
  if (str(i) == 'validation'):
     checker = True
  else:
     checker = False   
    
  #loop through each category
  j = 0
  while (j < 20):
    
    #break if 13+ videos are processed from the validation set
    if(j > 9 and checker == True):
      break
    
    #count the number of frames in each video
    path = '/content/drive/My Drive/Fall Detection/' + str(i) + '/testdata/video' + str(j)
    list = os.listdir(path) 
    numframes = len(list)
    
    #read in frames and determine beginning and ending frames of the fall
    filename = '/content/drive/My Drive/Fall Detection/' + str(i) + '/Annotation_files/video (' + str(j+1) + ').txt'
    dummydata = pd.read_csv(filename, sep=" ", header=None, nrows=2)
    frame1 = dummydata[0][0] 
    frame2 = dummydata[0][1] 
    frame1 = int(frame1) - 5
    frame2 = int(frame2) + 30
    
    #change directory to jsondata folder
    path = '/content/drive/My Drive/Fall Detection/' + str(i) + "/jsondata/video" + str(j)
    os.chdir(path)
    print("Video " + str(j) + ": Processing " + str(numframes) + " frames.")
    
    
    #add all frames into array
    k = 0
    vidarray = []
    
    while (k < numframes):
      with open('frame' + str(k) + '_keypoints.json') as f:  
        try:
          data = json.loads(f.read())
          list = data[0]['people'][0]['pose_keypoints_2d']
          successfulposes +=1
        except:
          list = np.zeros(75)
          nullposes +=1
      vidarray.append(list)
      k += 1
      
    videoX = np.array(vidarray)
    
    #split video array into sets for fall, walking, and afterfall
    FallSet = videoX[frame1:frame2,:]
    WalkSet = videoX[0:frame1-1,:]
    AfterFallSet = videoX[frame2+1:len(videoX),:]  
 
    #print(len(FallSet))
    #print(len(WalkSet))
    #print(len(AfterFallSet))
    
    #capture falls
    for _ in range(A):
      idx = np.random.randint(0,len(FallSet)-B)
      if (checker == False):
        X_train[trainframecount,:,:] = FallSet[idx:idx+B,:]
        Y_train[trainframecount] = 0
        trainframecount += 1
      else:
        X_test[testframecount,:,:] = FallSet[idx:idx+B,:]
        Y_test[testframecount] = 0
        testframecount += 1
      
    #capture walking
    for _ in range(A):
      idx = np.random.randint(0,len(WalkSet)-B)
      if (checker == False):
        X_train[trainframecount,:,:] = WalkSet[idx:idx+B,:]
        Y_train[trainframecount] = 1
        trainframecount += 1
      else:
        X_test[testframecount,:,:] = WalkSet[idx:idx+B,:]
        Y_test[testframecount] = 1
        testframecount += 1 
    
    j += 1   

#reshape Y data to categorical
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

#normalize input
X_train = X_train/X_train.max()
X_test = X_test/X_test.max()

print("Finished creating input array.")
print("Total train inputs: " + str(trainframecount))
print("Total test inputs: " +  str(testframecount))

print("X train: " + str(X_train.shape))
print("Y train: " + str(Y_train.shape))
print("X test: " + str(X_test.shape))
print("Y test: " + str(Y_test.shape))
print("null poses: " + str(nullposes))
print("successful poses:" + str(successfulposes))

Entering folder Coffee_room_01, which has 20 videos to process
Video 0: Processing 157 frames.
Video 1: Processing 306 frames.
Video 2: Processing 304 frames.
Video 3: Processing 207 frames.
Video 4: Processing 181 frames.
Video 5: Processing 239 frames.
Video 6: Processing 174 frames.
Video 7: Processing 258 frames.
Video 8: Processing 206 frames.
Video 9: Processing 362 frames.
Video 10: Processing 483 frames.
Video 11: Processing 182 frames.
Video 12: Processing 244 frames.
Video 13: Processing 176 frames.
Video 14: Processing 140 frames.
Video 15: Processing 177 frames.
Video 16: Processing 269 frames.
Video 17: Processing 213 frames.
Video 18: Processing 444 frames.
Video 19: Processing 182 frames.
Entering folder Home_01, which has 25 videos to process
Video 0: Processing 264 frames.
Video 1: Processing 240 frames.
Video 2: Processing 240 frames.
Video 3: Processing 240 frames.
Video 4: Processing 192 frames.
Video 5: Processing 192 frames.
Video 6: Processing 216 frames.
Video 7

# Train Model1, LSTM Network 

In [8]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten, Activation
from keras.layers.recurrent import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.convolutional import MaxPooling1D, MaxPooling2D
from tensorflow import keras
from keras.utils import to_categorical

model1 = Sequential()
model1.add(LSTM(100, input_shape=(B,75)))
model1.add(Dense(100, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(2, activation='softmax'))
print(model1.summary())
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model1.fit(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test))

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


E0711 08:09:28.051169 140174518364032 ultratb.py:152] Internal Python error in the inspect module.
Below is the traceback from this internal error.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100)               70400     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 202       
Total params: 80,702
Trainable params: 80,702
Non-trainable params: 0
_________________________________________________________________
None
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-de2408e3074b>", line 19, in <module>
    histo

ValueError: ignored

#Train Model2, Conv1D + LSTM Network

In [11]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten, Activation
from keras.layers.recurrent import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D, MaxPooling2D
from tensorflow import keras
from keras.utils import to_categorical

X_train = X_train.reshape(TotalTrain*A*2,5,6,75)
X_test = X_test.reshape(TotalTest*A*2,5,6,75)

model2 = Sequential()

model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(5,6,75)))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model2.add(TimeDistributed(Dropout(0.5)))
model2.add(TimeDistributed(MaxPooling1D(pool_size=1)))
model2.add(TimeDistributed(Flatten()))

model2.add(LSTM(100))
model2.add(Dropout(0.5))
model2.add(Dense(100, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(2, activation='softmax'))
print(model2.summary())
#model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model2.fit(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test))

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

E0711 08:10:57.493880 140174518364032 ultratb.py:152] Internal Python error in the inspect module.
Below is the traceback from this internal error.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_16 (TimeDis (None, 5, 4, 64)          14464     
_________________________________________________________________
time_distributed_17 (TimeDis (None, 5, 2, 64)          12352     
_________________________________________________________________
time_distributed_18 (TimeDis (None, 5, 2, 64)          0         
_________________________________________________________________
time_distributed_19 (TimeDis (None, 5, 2, 64)          0         
_________________________________________________________________
time_distributed_20 (TimeDis (None, 5, 128)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               91600     
_________________________________________________________________
dropout_13 (Dropout)         (None, 100)               0         
__________

RuntimeError: ignored

# **Test Models**


**Load in Test Data and test accuracy of both models**



---




In [0]:
def loaddata(testnumber):

  os.chdir('/content/drive/My Drive/Fall Detection')

  #count successful poses
  nullposes = 0
  successfulposes = 0

  #total videos
  Videos = 10

  #number of samples per activity (walking, falling, laying down)
  A = 20
  #of frames per sample (timesteps)
  B = 30

  X = np.zeros((Videos*A*2,B,75)) #creating an empty array for image json data
  Y = np.zeros((Videos*A*2,1)) #create empty array for y data

  #initialize counters
  trainframecount = 0
  testframecount = 0

  i = str(testnumber)

  #count the number of videos in category
  path = '/content/drive/My Drive/Fall Detection/test/Annotation_files'
  list = os.listdir(path)
  numvideos = len(list)

  #change into annotations file folder
  #print("Entering folder " + str(i) + ", which has " + str(numvideos) + " videos to process")
  os.chdir(path)
  
  #loop through each category
  j = 0
  while (j < numvideos):
    
    #count the number of frames in each video
    path = '/content/drive/My Drive/Fall Detection/' + str(i) + '/testdata/video' + str(j)
    list = os.listdir(path) 
    numframes = len(list)
    
    #read in frames and determine beginning and ending frames of the fall
    filename = '/content/drive/My Drive/Fall Detection/' + str(i) + '/Annotation_files/video (' + str(j+1) + ').txt'
    dummydata = pd.read_csv(filename, sep=" ", header=None, nrows=2)
    frame1 = dummydata[0][0] 
    frame2 = dummydata[0][1] 
    frame1 = int(frame1) - 5
    frame2 = int(frame2) + 30
    
    #change directory to jsondata folder
    path = '/content/drive/My Drive/Fall Detection/' + str(i) + "/jsondata/video" + str(j)
    os.chdir(path)
    #print("Video " + str(j) + ": Processing " + str(numframes) + " frames.")
    
    
    #add all frames into array
    k = 0
    vidarray = []
    
    while (k < numframes):
      with open('frame' + str(k) + '_keypoints.json') as f:  
        try:
          data = json.loads(f.read())
          list = data[0]['people'][0]['pose_keypoints_2d']
          successfulposes +=1
        except:
          list = np.zeros(75)
          nullposes +=1
      vidarray.append(list)
      k += 1
      
    videoX = np.array(vidarray)
    
    #split video array into sets for fall, walking, and afterfall
    FallSet = videoX[frame1:frame2,:]
    WalkSet = videoX[0:frame1-1,:]
    AfterFallSet = videoX[frame2+1:len(videoX),:]  
 
    #capture falls
    for _ in range(A):
      idx = np.random.randint(0,len(FallSet)-B)
      X[testframecount,:,:] = FallSet[idx:idx+B,:]
      Y[testframecount,:] = 0
      testframecount += 1
      
    #capture walking
    for _ in range(A):
      idx = np.random.randint(0,len(WalkSet)-B)
      X[testframecount,:,:] = WalkSet[idx:idx+B,:]
      Y[testframecount,:] = 1
      testframecount += 1 
    
    j += 1   
  
  #reshape Y data to categorical
  Y = to_categorical(Y)

  #normalize input
  X = X /X.max()
  
  return X, Y

'''
print("Finished creating input array.")
print("Total train inputs: " + str(trainframecount))
print("Total test inputs: " +  str(testframecount))

print("X train: " + str(X.shape))
print("Y train: " + str(Y.shape))
print("X test: " + str(X.shape))
print("Y test: " + str(Y.shape))
print("null poses: " + str(nullposes))
print("successful poses:" + str(successfulposes))
'''

from numpy import mean
from numpy import std

# repeat experiment
scores1 = []
scores2 = []
scores3 = []
scores4 = []

for r in range(20):
  X,Y = loaddata('test')
  accuracy1 = model1.evaluate(X, Y, verbose = 0)
  accuracy1 = accuracy1[1] * 100.0000
  print(accuracy1)
  scores1.append(accuracy1)

for r in range(20):
  X,Y = loaddata('test')
  X = X.reshape(Videos*A*2,5,6,75)
  accuracy2 = model2.evaluate(X, Y, verbose = 0)
  accuracy2 = accuracy2[1] * 100.0000
  print(accuracy2)
  scores2.append(accuracy2)
  
for r in range(20):
  X,Y = loaddata('test2')
  accuracy3 = model1.evaluate(X, Y, verbose = 0)
  accuracy3 = accuracy3[1] * 100.0000
  print(accuracy3)
  scores3.append(accuracy3)

for r in range(20):
  X,Y = loaddata('test2')
  X = X.reshape(Videos*A*2,5,6,75)
  accuracy4 = model2.evaluate(X, Y, verbose = 0)
  accuracy4 = accuracy4[1] * 100.0000
  print(accuracy4)
  scores4.append(accuracy4)

m1, s1 = mean(scores1), std(scores1)
m2, s2 = mean(scores2), std(scores2)
m3, s3 = mean(scores3), std(scores3)
m4, s4 = mean(scores4), std(scores4)
print('Accuracy 1, familiar data: %.3f%% (+/-%.3f)' % (m1, s1))
print('Accuracy 2, familiar data: %.3f%% (+/-%.3f)' % (m2, s2))
print('Accuracy 1, nonfamilar data: %.3f%% (+/-%.3f)' % (m3, s3))
print('Accuracy 2, nonfamilar data: %.3f%% (+/-%.3f)' % (m4, s4))

97.25
95.5
95.25
96.5
96.25
95.75
95.5
96.0
96.75
96.5
96.25
96.75
95.5
95.75
96.5
95.5
97.0
97.25
96.5
95.25


NameError: ignored